# Dynamic mode decomposition on two mixed spatial signals

This tutorial was initially published ([source](https://github.com/dynamicslab/pykoopman/blob/master/docs/tutorial_dmd_separating_two_mixed_signals_400d_system.ipynb)) in the the [pykoopman](https://github.com/dynamicslab/pykoopman) project, which is MIT licensed. A copyright notice is available in datafold's [LICENSED_bundled]() file (see file or project for full license details). The notebook is adapted for datafold. 

We utilize dynamic mode decomposition (DMD) on a linear spatiotemporal system. This system is formed by combining two mixed spatiotemporal signals, as demonstrated in example 1.4 of

> Kutz, J. Nathan, et al. Dynamic mode decomposition: data-driven modeling of complex systems. Society for Industrial and Applied Mathematics (SIAM), 2016.

The system consists of two additive signals:

$$f(x,t) = f_1(x,t) + f_2(x,t)$$

with

$$
\begin{aligned}
    f_1(x,t) &= \mathrm{sech}(x+3) e^{j2.3t},\\
    f_2(x,t) &= 2\,\mathrm{sech}(x)\,\mathrm{tanh}(x) e^{j2.8t}.
\end{aligned}
$$

The two separate signals exhibit frequencies: $\omega_1 = 2.3$ and $\omega_2 = 2.8$, each possessing a unique spatial configuration.

We import datafold's ``TSCDataFrame`` for a time series data structure and ``DMDStandard`` class for the DMD method. The other packages are required for plotting and matrix manipulation.

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
from matplotlib import cm
from mpl_toolkits.mplot3d import Axes3D  # noqa: F401

from datafold import DMDStandard, TSCDataFrame

SPecify time and space discretization of the system

In [ ]:
time_values = np.linspace(0, 4 * np.pi, 200)  # time array for solution
dt = time_values[1] - time_values[0]  # delta time
space_values = np.linspace(-10, 10, 400)
[Xgrid, Tgrid] = np.meshgrid(space_values, time_values)

Define helper function, hyperbolic secant

In [ ]:
def sech(x):
    return 1.0 / np.cosh(x)

Generate the training data from the two spatiotemporal signals

In [ ]:
omega1 = 2.3
omega2 = 2.8
f1 = np.multiply(sech(Xgrid + 3), np.exp(1j * omega1 * Tgrid))
f2 = np.multiply(
    np.multiply(sech(Xgrid), np.tanh(Xgrid)), 2 * np.exp(1j * omega2 * Tgrid)
)
f = TSCDataFrame.from_array(
    f1 + f2,
    time_values=time_values,
    feature_names=[f"x{i}" for i in range(len(space_values))],
)
f

In [ ]:
def plot_dynamics(space_grid, time_grid, f, fig=None, title="", subplot=111):
    if fig is None:
        fig = plt.figure(figsize=(12, 4))

    time_ticks = np.array([0, 1 * np.pi, 2 * np.pi, 3 * np.pi, 4 * np.pi])
    time_labels = ("0", r"$\pi$", r"$2\pi$", r"$3\pi$", r"$4\pi$")

    ax = fig.add_subplot(subplot, projection="3d")
    ax.plot_surface(space_grid, time_grid, f, rstride=1)
    ax.contourf(space_grid, time_grid, f, zdir="z", offset=-1.5, cmap=cm.ocean)
    ax.set(
        xlabel=r"$x$",
        ylabel=r"$t$",
        title=title,
        yticks=time_ticks,
        yticklabels=time_labels,
        xlim=(-10, 10),
        zlim=(-1.5, 1),
    )
    ax.autoscale(enable=True, axis="y", tight=True)

In [ ]:
fig = plt.figure(figsize=(12, 4))
fig.suptitle("Spatiotemporal dynamics of mixed signal")

plot_dynamics(Xgrid, Tgrid, f1.real, fig=fig, title=r"$f_1(x,t)$", subplot=131)
plot_dynamics(Xgrid, Tgrid, f2.real, fig=fig, title=r"$f_2(x,t)$", subplot=132)
plot_dynamics(
    Xgrid,
    Tgrid,
    f.to_numpy().real,
    fig=fig,
    title=r"$f(x, t) = f_1(x,t) + f_2(x,t)$",
    subplot=133,
)

Instantiate and fit DMD model on training data to approximate system from data.

In [ ]:
dmd = DMDStandard(rank=2)
dmd.fit(f, store_system_matrix=True)

In [ ]:
K = dmd.system_matrix_  # approximate Koopman operator

# Ingestigate eigenvalues of the Koopman matrix
evals, evecs = np.linalg.eig(K)
# convert the eigenvalues of the discrete system to the continuous counterpart
evals_cont = np.log(evals) / dt

fig = plt.figure(figsize=(4, 4))
ax = fig.add_subplot(111)
ax.plot([0, 0], [omega1, omega2], "rs", label="true", markersize=10)
ax.plot(evals_cont.real, evals_cont.imag, "bo", label="estimated", markersize=5)
ax.grid()
ax.set_xlim([-1, 1])
ax.set_ylim([2, 3])
plt.legend()
plt.xlabel(r"$Re(\lambda)$")
plt.ylabel(r"$Im(\lambda)$");

Use the DMD model to reconstruct the training data, where the first sample is the initial condition:

In [ ]:
f_predicted = dmd.reconstruct(f)

fig = plt.figure(figsize=(8, 4))
fig.suptitle("DMD simulation")

plot_dynamics(Xgrid, Tgrid, f.to_numpy().real, fig=fig, title=r"$f(x, t)$", subplot=121)
plot_dynamics(
    Xgrid, Tgrid, f_predicted.to_numpy().real, fig=fig, title="DMD", subplot=122
)

In [ ]:
# mean absolute error
mae_error = np.abs((f_predicted - f).to_numpy().real).mean(axis=1)

fig, ax = plt.subplots(figsize=[5, 2.5])
ax.set_xlabel("time")
ax.set_ylabel("error")
ax.grid()
ax.plot(f_predicted.time_values(), mae_error, c="red");

We see that the reconstruction error for is fairly low (still within machine precision).